# Creation of the portfolios

This part will create a csv file with the characteristics of different possible portfolios:

Each lines will represent a different portfolio, with, 

* for eah assets (oil, gas, wheat), the weight of this asset in the portfolio. Weights changes by range of 5%.
* The total return of the portfolio
* The yearly average return
* The volatility of the portfolio
* The sharpe ration
* The sortino ration

One Csv will be created for each time range (6 months, 1 year, 5 years, all)

In [1]:
import numpy as np
import pandas as pd

In [2]:
def create_portfolios_dist() ->pd.DataFrame:
    """ 
    create and return a dataframe with the distributions of different portfolios
    """
    assets = ['Crude Oil', 'Natural Gas', 'Chicago Wheat']
    n_assets = len(assets)
    weight_steps = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]
    n_weights = len(weight_steps)

    weights = []
    for i in range(n_weights):
        for j in range(n_weights):
            for k in range(n_weights):
                if weight_steps[i] + weight_steps[j] + weight_steps[k] == 1.0:
                    weights.append([weight_steps[i], weight_steps[j], weight_steps[k]])

    # Create dataframes
    df = pd.DataFrame(weights, columns=assets)
    return df

In [3]:
# create portfolios distribution
df = create_portfolios_dist()

# import and add price data
prices = pd.read_csv('Prices.csv')
prices['Date'] = pd.to_datetime(prices['Date'])

In [4]:
def asset_return(asset: pd.DataFrame) -> float:
    return (asset.iloc[-1] - asset.iloc[0]) / asset.iloc[0]

In [5]:
start_date = [prices['Date'].min(), prices['Date'].max() - pd.DateOffset(years=10), prices['Date'].max() - pd.DateOffset(years=5),
              prices['Date'].max() - pd.DateOffset(years=2), prices['Date'].max() - pd.DateOffset(years=1)
              ]
names_csv = ['portfolios_all.csv', 'portfolios_10y.csv', 'portfolios_5y.csv', 'portfolios_2y.csv', 'portfolios_1y.csv']

for i in range(0,len(start_date)):

    #at first copy the dataframes, to make sure we won't have 'conflict' after
    df_all = df.copy()
    prices_all = prices.copy()

    #set the start date--------------------------------------------------------------------------------------------
    start = start_date[i]
    prices_all = prices_all[prices_all['Date'] >= start]

    #set total return-----------------------------------------------------------------------------------------------
    #compute total return for each assets
    return_gas = asset_return(prices_all['Natural Gas'])
    return_oil = asset_return(prices_all['Crude Oil'])
    return_wheat = asset_return(prices_all['Chicago Wheat'])

    df_all['Total_return'] = df_all['Natural Gas'] * return_gas + df_all['Crude Oil'] * return_oil + df_all['Chicago Wheat'] * return_wheat
    #add 3 columns=> easier for tableau
    df_all['return_gas'] = return_gas
    df_all['return_oil'] = return_oil
    df_all['return_wheat'] = return_wheat

    #set the avg annual return-----------------------------------------------------------------------------------
    df_all['Avg_anual_return'] = ((
                             prices_all['Crude Oil'].pct_change().mean() * df_all['Crude Oil'] + 
                             prices_all['Natural Gas'].pct_change().mean() * df_all['Natural Gas'] + 
                             prices_all['Chicago Wheat'].pct_change().mean() * df_all['Chicago Wheat'] 
                             )+1) ** (252) -1
    #add 3 columns
    df_all['avg_return_gas'] = (prices_all['Natural Gas'].pct_change().mean() + 1) ** (252) -1
    df_all['avg_return_oil'] = (prices_all['Crude Oil'].pct_change().mean() + 1) ** (252) -1
    df_all['avg_return_wheat'] = (prices_all['Chicago Wheat'].pct_change().mean() + 1) ** (252) -1

    # set the volatility------------------------------------------------------------------------------------------
    matrice = np.log(prices_all[['Crude Oil', 'Natural Gas', 'Chicago Wheat']].shift(1) / 
                     prices_all[['Crude Oil', 'Natural Gas', 'Chicago Wheat']]).cov().values

    for index, row in df_all.iterrows():
        vecteur = np.array([row['Crude Oil'], row['Natural Gas'], row['Chicago Wheat']])
        produit = np.dot(vecteur, np.dot(matrice, vecteur.T))
        df_all.at[index, 'Volatility'] = np.sqrt(produit) * np.sqrt(252)

    #add 3 columns (for tableau)
    df_all['vol_gas'] = prices_all['Natural Gas'].pct_change().var()**(1/2)
    df_all['vol_wheat'] = prices_all['Chicago Wheat'].pct_change().var()**(1/2)
    df_all['vol_oil'] = prices_all['Crude Oil'].pct_change().var()**(1/2)


    #set sortino ratio-------------------------------------------------------------------------------------------
    treasury_bill_year_return = 0.0029
    df_all['Sharpe_ratio'] = (df_all['Avg_anual_return'] - treasury_bill_year_return) / df_all['Volatility']

    df_all.to_csv(names_csv[i], index=False)
    display(df_all)


,Crude Oil,Natural Gas,Chicago Wheat,Total_return,return_gas,return_oil,return_wheat,Avg_anual_return,avg_return_gas,avg_return_oil,avg_return_wheat,Volatility,vol_gas,vol_wheat,vol_oil,Sharpe_ratio
0,0.00,0.00,1.00,2.155378,-0.074457,1.404434,2.155378,0.109794,0.173065,0.140077,0.109794,0.324328,0.036305,0.02051,0.026984,0.329585
1,0.00,0.05,0.95,2.043887,-0.074457,1.404434,2.155378,0.112875,0.173065,0.140077,0.109794,0.311383,0.036305,0.02051,0.026984,0.353183
2,0.00,0.10,0.90,1.932395,-0.074457,1.404434,2.155378,0.115965,0.173065,0.140077,0.109794,0.301245,0.036305,0.02051,0.026984,0.375325
3,0.00,0.15,0.85,1.820903,-0.074457,1.404434,2.155378,0.119063,0.173065,0.140077,0.109794,0.294205,0.036305,0.02051,0.026984,0.394837
4,0.00,0.20,0.80,1.709411,-0.074457,1.404434,2.155378,0.122170,0.173065,0.140077,0.109794,0.290489,0.036305,0.02051,0.026984,0.410584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,0.90,0.05,0.05,1.368036,-0.074457,1.404434,2.155378,0.140168,0.173065,0.140077,0.109794,0.395893,0.036305,0.02051,0.026984,0.346731
221,0.90,0.10,0.00,1.256545,-0.074457,1.404434,2.155378,0.143333,0.173065,0.140077,0.109794,0.401259,0.036305,0.02051,0.026984,0.349982
222,0.95,0.00,0.05,1.441981,-0.074457,1.404434,2.155378,0.138543,0.173065,0.140077,0.109794,0.410805,0.036305,0.02051,0.026984,0.330189
223,0.95,0.05,0.00,1.330489,-0.074457,1.404434,2.155378,0.141704,0.173065,0.140077,0.109794,0.414249,0.036305,0.02051,0.026984,0.335074


,Crude Oil,Natural Gas,Chicago Wheat,Total_return,return_gas,return_oil,return_wheat,Avg_anual_return,avg_return_gas,avg_return_oil,avg_return_wheat,Volatility,vol_gas,vol_wheat,vol_oil,Sharpe_ratio
0,0.00,0.00,1.00,0.017995,0.335422,-0.125898,0.017995,0.048943,0.202847,0.101994,0.048943,0.302845,0.035395,0.019189,0.029614,0.152035
1,0.00,0.05,0.95,0.033866,0.335422,-0.125898,0.017995,0.056150,0.202847,0.101994,0.048943,0.290189,0.035395,0.019189,0.029614,0.183501
2,0.00,0.10,0.90,0.049738,0.335422,-0.125898,0.017995,0.063406,0.202847,0.101994,0.048943,0.280439,0.035395,0.019189,0.029614,0.215755
3,0.00,0.15,0.85,0.065609,0.335422,-0.125898,0.017995,0.070712,0.202847,0.101994,0.048943,0.273907,0.035395,0.019189,0.029614,0.247573
4,0.00,0.20,0.80,0.081480,0.335422,-0.125898,0.017995,0.078068,0.202847,0.101994,0.048943,0.270824,0.035395,0.019189,0.029614,0.277552
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,0.90,0.05,0.05,-0.095638,0.335422,-0.125898,0.017995,0.104104,0.202847,0.101994,0.048943,0.430091,0.035395,0.019189,0.029614,0.235308
221,0.90,0.10,0.00,-0.079766,0.335422,-0.125898,0.017995,0.111688,0.202847,0.101994,0.048943,0.433564,0.035395,0.019189,0.029614,0.250917
222,0.95,0.00,0.05,-0.118704,0.335422,-0.125898,0.017995,0.099279,0.202847,0.101994,0.048943,0.449983,0.035395,0.019189,0.029614,0.214185
223,0.95,0.05,0.00,-0.102832,0.335422,-0.125898,0.017995,0.106831,0.202847,0.101994,0.048943,0.451684,0.035395,0.019189,0.029614,0.230096


,Crude Oil,Natural Gas,Chicago Wheat,Total_return,return_gas,return_oil,return_wheat,Avg_anual_return,avg_return_gas,avg_return_oil,avg_return_wheat,Volatility,vol_gas,vol_wheat,vol_oil,Sharpe_ratio
0,0.00,0.00,1.00,0.826990,0.464332,0.329468,0.82699,0.195248,0.341852,0.243807,0.195248,0.339415,0.041958,0.021559,0.035734,0.566705
1,0.00,0.05,0.95,0.808857,0.464332,0.329468,0.82699,0.202184,0.341852,0.243807,0.195248,0.325401,0.041958,0.021559,0.035734,0.612427
2,0.00,0.10,0.90,0.790724,0.464332,0.329468,0.82699,0.209160,0.341852,0.243807,0.195248,0.315004,0.041958,0.021559,0.035734,0.654785
3,0.00,0.15,0.85,0.772591,0.464332,0.329468,0.82699,0.216176,0.341852,0.243807,0.195248,0.308591,0.041958,0.021559,0.035734,0.691129
4,0.00,0.20,0.80,0.754458,0.464332,0.329468,0.82699,0.223233,0.341852,0.243807,0.195248,0.306412,0.041958,0.021559,0.035734,0.719075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,0.90,0.05,0.05,0.361088,0.464332,0.329468,0.82699,0.246052,0.341852,0.243807,0.195248,0.519727,0.041958,0.021559,0.035734,0.467845
221,0.90,0.10,0.00,0.342955,0.464332,0.329468,0.82699,0.253281,0.341852,0.243807,0.195248,0.523125,0.041958,0.021559,0.035734,0.478626
222,0.95,0.00,0.05,0.354344,0.464332,0.329468,0.82699,0.241333,0.341852,0.243807,0.195248,0.544340,0.041958,0.021559,0.035734,0.438022
223,0.95,0.05,0.00,0.336212,0.464332,0.329468,0.82699,0.248535,0.341852,0.243807,0.195248,0.545671,0.041958,0.021559,0.035734,0.450152


,Crude Oil,Natural Gas,Chicago Wheat,Total_return,return_gas,return_oil,return_wheat,Avg_anual_return,avg_return_gas,avg_return_oil,avg_return_wheat,Volatility,vol_gas,vol_wheat,vol_oil,Sharpe_ratio
0,0.00,0.00,1.00,0.236052,0.847647,0.658264,0.236052,0.212643,0.874226,0.405465,0.212643,0.416794,0.051205,0.026563,0.026399,0.503230
1,0.00,0.05,0.95,0.266631,0.847647,0.658264,0.236052,0.239353,0.874226,0.405465,0.212643,0.400308,0.051205,0.026563,0.026399,0.590678
2,0.00,0.10,0.90,0.297211,0.847647,0.658264,0.236052,0.266650,0.874226,0.405465,0.212643,0.388159,0.051205,0.026563,0.026399,0.679488
3,0.00,0.15,0.85,0.327791,0.847647,0.658264,0.236052,0.294545,0.874226,0.405465,0.212643,0.380762,0.051205,0.026563,0.026399,0.765950
4,0.00,0.20,0.80,0.358371,0.847647,0.658264,0.236052,0.323052,0.874226,0.405465,0.212643,0.378396,0.051205,0.026563,0.026399,0.846076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,0.90,0.05,0.05,0.646623,0.847647,0.658264,0.236052,0.415371,0.874226,0.405465,0.212643,0.392169,0.051205,0.026563,0.026399,1.051766
221,0.90,0.10,0.00,0.677203,0.847647,0.658264,0.236052,0.446527,0.874226,0.405465,0.212643,0.396479,0.051205,0.026563,0.026399,1.118917
222,0.95,0.00,0.05,0.637154,0.847647,0.658264,0.236052,0.395136,0.874226,0.405465,0.212643,0.407096,0.051205,0.026563,0.026399,0.963498
223,0.95,0.05,0.00,0.667734,0.847647,0.658264,0.236052,0.425849,0.874226,0.405465,0.212643,0.407376,0.051205,0.026563,0.026399,1.038226


,Crude Oil,Natural Gas,Chicago Wheat,Total_return,return_gas,return_oil,return_wheat,Avg_anual_return,avg_return_gas,avg_return_oil,avg_return_wheat,Volatility,vol_gas,vol_wheat,vol_oil,Sharpe_ratio
0,0.00,0.00,1.00,0.015710,0.256669,0.042473,0.01571,0.159032,1.045603,0.170187,0.159032,0.512815,0.063308,0.032767,0.030256,0.304462
1,0.00,0.05,0.95,0.027758,0.256669,0.042473,0.01571,0.192463,1.045603,0.170187,0.159032,0.493659,0.063308,0.032767,0.030256,0.383997
2,0.00,0.10,0.90,0.039806,0.256669,0.042473,0.01571,0.226855,1.045603,0.170187,0.159032,0.479782,0.063308,0.032767,0.030256,0.466785
3,0.00,0.15,0.85,0.051854,0.256669,0.042473,0.01571,0.262234,1.045603,0.170187,0.159032,0.471649,0.063308,0.032767,0.030256,0.549846
4,0.00,0.20,0.80,0.063902,0.256669,0.042473,0.01571,0.298629,1.045603,0.170187,0.159032,0.469559,0.063308,0.032767,0.030256,0.629803
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,0.90,0.05,0.05,0.051845,0.256669,0.042473,0.01571,0.202786,1.045603,0.170187,0.159032,0.452921,0.063308,0.032767,0.030256,0.441326
221,0.90,0.10,0.00,0.063893,0.256669,0.042473,0.01571,0.237474,1.045603,0.170187,0.159032,0.458810,0.063308,0.032767,0.030256,0.511266
222,0.95,0.00,0.05,0.041135,0.256669,0.042473,0.01571,0.169626,1.045603,0.170187,0.159032,0.467593,0.063308,0.032767,0.030256,0.356563
223,0.95,0.05,0.00,0.053183,0.256669,0.042473,0.01571,0.203362,1.045603,0.170187,0.159032,0.468284,0.063308,0.032767,0.030256,0.428077
